#Imports


In [ ]:
!pip install ufal.udpipe pymorphy2 natasha transformers torch sacremoses

In [2]:
import xml.etree.ElementTree as ET
import ufal.udpipe
import re
import pymorphy2
from natasha import Segmenter, NewsEmbedding, NewsSyntaxParser, Doc
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, T5ForConditionalGeneration, T5Tokenizer
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"

In [9]:
df = pd.read_csv('/content/texts.csv')
print(df.head())

                                                text
0  I've been meaning to write for ages and finall...
1  I'm writing this review to give you a heads up...
2  Hi Meena, Thank you so much for offering to ho...
3  Hi Helga,\nI've been meaning to write to you f...
4  Dear Professor Henley,\nI am writing to inform...


# Translators

##opus-mt-en-ru

In [12]:
tokenizer_opus = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-ru")
model_opus = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-en-ru").to(device)

In [10]:
def translate_text(text):
    input_ids = tokenizer_opus(text, return_tensors="pt").input_ids.to(device)
    output_ids = model_opus.generate(input_ids)
    translated_text = tokenizer_opus.decode(output_ids[0], skip_special_tokens=True)
    return translated_text

In [11]:
text_opus_mt_en_ru = translate_text(df['text'][3])
print(text_opus_mt_en_ru)

Здравствуйте, Хельга, я давно собирался писать вам, так что не волнуйтесь! Как прошли ваши экзамены? Когда вы узнаете результаты? Я уверен, что вы сделали блестяще! Что касается меня, то я был на новой работе три месяца к концу следующей недели, так что я чувствую себя более комфортно. Сначала я чувствовал, что не имею понятия, что я делаю, но теперь я понимаю, что это нормально. Нам нужно было многому научиться - на самом деле, и мне пришлось привыкнуть к мысли, что я не знаю всего. Я привыкла работать допоздна и по выходным, но я медленно вхожу в нормальную рутину. Что означает, что я бы с удовольствием приехала и навестила вас! Нам действительно нужно наверстать упущенное! Я не могу поверить, что мы не видели друг друга после свадьбы Карла. Как звучит в следующем месяце? В любом случае, мне лучше вернуться на работу. Поздравляю вас на новой квартире.


##utrobinmv/t5_translate_en_ru_zh_large_1024

In [ ]:
model_name = 'utrobinmv/t5_translate_en_ru_zh_large_1024'
model_utrobinmv = T5ForConditionalGeneration.from_pretrained(model_name)
model_utrobinmv.to(device)
tokenizer_utrobinmv = T5Tokenizer.from_pretrained(model_name)

In [8]:
prefix = 'translate to ru: '
src_text = prefix + df['text'][2]

input_ids = tokenizer_utrobinmv(src_text, return_tensors="pt")

In [9]:
generated_tokens = model_utrobinmv.generate(**input_ids.to(device))

text_t5 = tokenizer_utrobinmv.batch_decode(generated_tokens, skip_special_tokens=True)

In [10]:
text_t5 = text_t5[0]
print(text_t5)

Привет, Миена, спасибо большое за предложение сидеть дома для нас на следующей неделе. Я только сожалею, что мы не сможем догнать должным образом, пока мы не вернемся из нашей поездки. В любом случае, вот все, что я не успею сказать вам в субботу утром. Сигнальный код 7957. Не забудьте установить его, когда вы выходите и помните, чтобы выключить его, как это БУДЕТ! Кроликам нужно кормить один раз в день и ни при каких обстоятельствах они не должны быть разрешены выйти из своей клетки в саду, поскольку соседские кошки напали на них в прошлом. Пожалуйста, можете использовать стиральную машину в дневное время только потому, что она старая и шумная, а соседи жалуются на обратное? Вы не возражаете открыть все окна наверху на час утром, так как дом становится сырым в это время года? Это о нем! Вы найдете все необходимое в доме, и помогите себе во всем в холодильнике или шкафах. Не стесняйтесь позвонить или написать, если у вас есть какие-либо проблемы / вопросы. Увидимся кратко, чтобы переда

# Text formatting

## load UDPipe

In [14]:
model_path = "/content/russian-syntagrus-ud-2.5-191206.udpipe"
model = ufal.udpipe.Model.load(model_path)

In [15]:
pipeline = ufal.udpipe.Pipeline(model, "tokenize", ufal.udpipe.Pipeline.DEFAULT, ufal.udpipe.Pipeline.DEFAULT, 'matxin')

## Separation of text and preservation of punctuation marks

In [13]:
# text = text_t5
text = text_opus_mt_en_ru

In [16]:
punctuation_positions = [m.group() for m in re.finditer(r'[\n«»,.?!:()]+', text)]
parts = re.split(r'[«»,.?!:(!)]', text)
parts = [part.strip() for part in parts if part.strip()]

## Formatting functions

In [17]:
def word_dependence(text):
  emb = NewsEmbedding()
  segmenter = Segmenter()
  syntax_parser = NewsSyntaxParser(emb)
  doc = Doc(text)
  doc.segment(segmenter)
  doc.parse_syntax(syntax_parser)

  root = ''
  dependencies = {}
  info_words = {token.id: [token.rel, token.text] for token in doc.tokens}
  if len(doc.tokens) > 1:
    for token in doc.tokens:
      if token.rel == 'root':
        root = token.id
      dependents = [dep.id for dep in doc.tokens if dep.head_id == token.id]
      if dependents:
        dependencies[token.id] = dependents
    if root == '' and len(dependencies) == 1:
      root = next(iter(dependencies.keys()))
  else:
    token = doc.tokens[0]
    root = token.id
    dependencies = {token.id: []}
  return root, dependencies, info_words

In [18]:
def parse_node(node, result):
    result['1_' + node.attrib.get('ord')] = ['1_' + child.attrib.get('ord') for child in node]
    return {key: value for key, value in result.items() if value}

In [19]:
def extract_info(node):
  form = node.attrib.get('form')
  mi = node.attrib.get('mi')
  si = node.attrib.get('si')
  properties = mi.split('|')
  person = next((prop.split('=')[1] for prop in properties if prop.startswith('Person=')), '')
  gender = next((prop.split('=')[1] for prop in properties if prop.startswith('Gender=')), '')
  return [form, si, person, gender]

In [20]:
def result_words(text, text_info, root):
  if root == '':
    parsed_text = pipeline.process(text)
    root = ET.fromstring(parsed_text)
    res, words_info = {}, {}
    for sentence in root.findall('SENTENCE'):
      for node in sentence.findall('.//NODE'):
        res = parse_node(node, res)

    for sentence in root.findall('SENTENCE'):
      for node in sentence.findall('.//NODE'):
        if '1_' + node.attrib.get('ord') not in words_info:
          words_info['1_' + node.attrib.get('ord')] = []
        words_info['1_' + node.attrib.get('ord')] = extract_info(node)
    words_info = dict(sorted(words_info.items()))

    for key in words_info.keys():
      if words_info[key][1] == 'root':
        root = key

    for key in text_info.keys():
      text_info[key][0] = words_info[key][1]
      text_info[key].extend([words_info[key][-2], words_info[key][-1]])

    return root, text_info, res

  else:
    parsed_text = pipeline.process(text)
    root_parse = ET.fromstring(parsed_text)
    words_info = {}
    for sentence in root_parse.findall('SENTENCE'):
      for node in sentence.findall('.//NODE'):
        if '1_' + node.attrib.get('ord') not in words_info:
          words_info['1_' + node.attrib.get('ord')] = []
        words_info['1_' + node.attrib.get('ord')] = extract_info(node)
    words_info = dict(sorted(words_info.items()))
    for key in text_info.keys():
      text_info[key].extend([words_info[key][-2], words_info[key][-1]])
    return text_info

In [21]:
def masc_to_femn(word):
  morph = pymorphy2.MorphAnalyzer()
  p = morph.parse(word)[0]
  fem_word = p.inflect({'femn'})
  if fem_word:
    fem_word = fem_word.word
    if word.istitle():
      fem_word = word[0] + fem_word[1:]
    return fem_word, 'femn'
  else:
    return word, 'Masc'

In [22]:
def dfs_forms(tree, root, info_nodes, len_sentence, punct, previous_change):
  flag = False
  if len_sentence == 1: # если часть предложения из 1 слова
    if info_nodes[root][2] in ['', '1'] and info_nodes[root][3] == 'Masc':
      info_nodes[root][1], info_nodes[root][-1] = masc_to_femn(info_nodes[root][1])
      flag = True
  else:
    nsubj = ''
    for key in tree[root]:
      if info_nodes[key][0] in ['nsubj', 'nsubj:pass'] and info_nodes[key][3] != 'Neut' and info_nodes[key][1].lower() != 'мы':
        nsubj = key
    if nsubj == '':
      for key in info_nodes.keys():
        if info_nodes[key][0] == 'nsubj' and info_nodes[key][1].lower() == 'я':
          nsubj = key
    if nsubj != '':
      if info_nodes[nsubj][3] != 'Masc' and info_nodes[nsubj][2] != '3': # Проход по зависимым словам от подлежащего
        if nsubj in tree.keys():
          for key in tree[nsubj]:
            if info_nodes[key][0] in ['obj', 'aux:pass', 'cop'] and info_nodes[key][3] in ['', 'Masc']:
              info_nodes[key][1], info_nodes[key][3] = masc_to_femn(info_nodes[key][1])
              flag = True
        if info_nodes[root][2] in ['', '1']: # Проход по зависимым словам от корня
          info_nodes[root][1], info_nodes[root][-1] = masc_to_femn(info_nodes[root][1])
          flag = True
          for key in tree[root]:
            if info_nodes[key][0] in ['obj', 'aux:pass', 'cop', 'acl'] and info_nodes[key][3] in ['', 'Masc']:
              info_nodes[key][1], info_nodes[key][3] = masc_to_femn(info_nodes[key][1])
              flag = True
    else: # Если нет подлежащего (подчиненная часть сложного предложения)
      if punct == ',' and previous_change:
        if info_nodes[root][2] in ['', '1'] and info_nodes[root][3] == 'Masc':
          info_nodes[root][1], info_nodes[root][-1] = masc_to_femn(info_nodes[root][1])
          flag = True
          for key in tree[root]:
            if info_nodes[key][0] in ['obj', 'aux:pass'] and info_nodes[key][3] in ['', 'Masc']:
              info_nodes[key][1], info_nodes[key][3] = masc_to_femn(info_nodes[key][1])
              flag = True
  return flag

In [23]:
def correct_form(part_sentence, punct, prev_change):
  root, depend, info = word_dependence(part_sentence)
  if not root:
    root, info, depend = result_words(part_sentence, info, root)
  else:
    info = result_words(part_sentence, info, root)

  previous = dfs_forms(depend, root, info, len(part_sentence.split()), punct, prev_change)
  stroka = ' '.join(info[key][1] for key in info.keys())

  return stroka, previous

# Starting the translation formatting

In [24]:
result = ''
for i in range(len(parts)):
  if i == 0:
    temp, prev_change = correct_form(parts[i], '', False)
  else:
    temp, prev_change = correct_form(parts[i], punctuation_positions[i-1], prev_change)
  if i != len(parts)-1:
    result += temp + punctuation_positions[i]
  else:
    if len(parts) != len(punctuation_positions):
      result += temp
    else:
      result += temp + punctuation_positions[i]
  if i != len(parts) - 1:
    result += ' '

In [25]:
print(text)
print(result)

Здравствуйте, Хельга, я давно собирался писать вам, так что не волнуйтесь! Как прошли ваши экзамены? Когда вы узнаете результаты? Я уверен, что вы сделали блестяще! Что касается меня, то я был на новой работе три месяца к концу следующей недели, так что я чувствую себя более комфортно. Сначала я чувствовал, что не имею понятия, что я делаю, но теперь я понимаю, что это нормально. Нам нужно было многому научиться - на самом деле, и мне пришлось привыкнуть к мысли, что я не знаю всего. Я привыкла работать допоздна и по выходным, но я медленно вхожу в нормальную рутину. Что означает, что я бы с удовольствием приехала и навестила вас! Нам действительно нужно наверстать упущенное! Я не могу поверить, что мы не видели друг друга после свадьбы Карла. Как звучит в следующем месяце? В любом случае, мне лучше вернуться на работу. Поздравляю вас на новой квартире.
Здравствуйте, Хельга, я давно собиралась писать вам, так что не волнуйтесь! Как прошли ваши экзамены? Когда вы узнаете результаты? Я у